In [1]:
import torch
torch.cuda.is_available()

False

In [2]:
 
 !pip install transformers einops accelerate langchain bitsandbytes sentence_transformers faiss-cpu gradio pypdf sentencepiece 

In [3]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline 

from transformers import AutoTokenizer 

from langchain.embeddings import HuggingFaceEmbeddings 

from langchain.document_loaders.csv_loader import CSVLoader 

from langchain.vectorstores import FAISS 

from langchain.chains import RetrievalQA 

import transformers 

import torch 

import pandas as pd

import gradio 

import textwrap 

In [4]:
model = "meta-llama/Llama-2-7b-chat-hf" 

tokenizer = AutoTokenizer.from_pretrained(model) 

 

pipeline = transformers.pipeline( 

 "text-generation", #task 

 model=model, 

tokenizer=tokenizer, 

 torch_dtype=torch.bfloat16, 

 trust_remote_code=True, 

device_map="auto", 

max_length=1000, 

do_sample=True, 

top_k=10, 

num_return_sequences=1, 

eos_token_id=tokenizer.eos_token_id 

) 

 

llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0}) 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cpu'}) 

In [6]:
data = pd.read_csv('C:\\Users\\Om\\PycharmProjects\\Agriculture Data Analyser\\Data\\Production_Crops_Livestock_E_Flags.csv')


In [7]:
from langchain_core.documents import Document

# Assuming "text" is the column containing text content in your DataFrame
documents = [Document(page_content=str(row[data.columns]), metadata={'id': idx}) for idx, row in data.iterrows()]
# documents = [Document(page_content=row[data.columns], metadata={'id': idx}) for idx, row in data.iterrows()]

# Now you can use the documents list to create the vectorstore
vectorstore = FAISS.from_documents(documents, embeddings)

In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type = "stuff",return_source_documents=True, retriever=vectorstore.as_retriever()) 

query = "What is the expantion of M?" 

result=chain(query) 

wrapped_text = textwrap.fill(result['result'], width=500) 

wrapped_text 
llm("HI?")

In [ ]:
import gradio as gr 

import pandas as pd 

 
  
def main(dataset,qs): 

        #df = pd.read_csv(dataset.name) 
        chain = RetrievalQA.from_chain_type(llm=llm, chain_type = "stuff",return_source_documents=False, retriever=vectorstore.as_retriever()) 

         #query = "What is the annual salary of Sophie Silva?" 

        result=chain(qs) 

        wrapped_text = textwrap.fill(result['result'], width=500) 
        return wrapped_text 

 

In [ ]:
def dataset_change(dataset): 

    global vectorstore 

    data = pd.read_csv(dataset.name) 

    # data = loader.load() 
    from langchain_core.documents import Document

    # Assuming "text" is the column containing text content in your DataFrame
    documents = [Document(page_content=str(row[data.columns]), metadata={'id': idx}) for idx, row in data.iterrows()]
    vectorstore = FAISS.from_documents(documents, embeddings)
    df = pd.read_csv(dataset.name) 

    return df.head(5) 

 

with gr.Blocks() as demo: 

    with gr.Row(): 

        with gr.Column(): 

            data = gr.File() 

            qs = gr.Text(label="Input Question") 

            submit_btn = gr.Button("Submit") 

        with gr.Column(): 

            answer = gr.Text(label="Output Answer") 

    with gr.Row(): 

        dataframe = gr.Dataframe() 

    submit_btn.click(main, inputs=[data,qs], outputs=[answer]) 

    data.change(fn=dataset_change,inputs = data,outputs=[dataframe]) 

    gr.Examples([["What is the Annual Salary of Theodore Dinh?"], ["What is the Department of Parker James?"]], inputs=[qs]) 

demo.launch(debug=True) 

 
